### Introduction

The objective of this example is to run the basic calibration steps on raw frames.
By the end of this example, we will know how to:

- Pre-process raw frames to remove the overscan section and extract the science section
- Remove cosmic rays from individual frames using the L.A. algorithm (Van Dokkum+2005)
- Produce calibrated bias, dark and pixelflat frames

Let's start with some imports and function definitions

In [17]:
import os
import itertools as it
import matplotlib.pyplot as plt

from lvmdrp.core.constants import LVM_UNAM_URL
from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata

from lvmdrp.functions import imageMethod

plt.style.use("seaborn-v0_8-talk")

### Downloading 2D simulations

Before we begin, we need to:
- Define paths to input and output data
- Download some data to run through the pipeline

In this case, we'll download 2D simulated data.

In [10]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.{ext}")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.{ext}")

In [11]:
# now we download the example data
fetch_example_data(url=LVM_UNAM_URL, name="sim_2d", dest_path=data_path)

[INFO]: example data already exists


In order to organize our simulated data and target specific type of frames better,
we extract relevant metadata from the frames and group them by image type, camera
and exposure time.

In [12]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "sim_2d"))
# group by imagetyp, camera and exptime
frames_table = frames_table.group_by(["mjd", "imagetyp", "spec", "camera", "expnum", "exptime"])
# define groups and unique categories in those groups
frames_groups = frames_table.groups
unique_groups = frames_groups.keys

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/frames_table.pkl'


We will start by processing calibration frames: bias, dark and pixelflats

Let's start with the biases for the blue channel of spectrograph #1...

In [19]:
mjds = np.unique(unique_groups["mjd"])
cameras = ["b", "r", "z"]
spec = 1
for mjd, camera in it.product(mjds, cameras):
    biases = frames_groups[(unique_groups["mjd"]==mjd)&(unique_groups["imagetyp"]=="bias")&(unique_groups["spec"]==f"sp{spec}")&(unique_groups["camera"]==f"{camera}{spec}")]
    biases.pprint()
    
    # let's pre-process the all bias frames
    pbias_paths = []
    for bias_frame in biases:
        bias_path = bias_frame["path"]
        bias_camera, bias_expnum = bias_frame["camera"], bias_frame["expnum"]

        pbias_path = out_main_path.format(kind="pbias", camera=bias_camera, expnum=bias_expnum, ext="fits")

        imageMethod.preprocRawFrame_drp(
            in_image=bias_path,
            out_image=pbias_path,
            subtract_overscan=True, unit="electron", compute_error=True,
        )
        pbias_paths.append(pbias_path)

    imageMethod.createMasterFrame_drp(
        in_images=pbias_paths,
        out_image=out_calib_path.format(mjd=mjd, kind="mbias", camera=bias_camera, exptime=int(bias_frame["exptime"]), ext="fits")
    )

imagetyp spec  mjd  camera  expnum  exptime                                           path                                         
-------- ---- ----- ------ -------- ------- ---------------------------------------------------------------------------------------
    bias  sp1 60023     b1 00000101     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-b1-00000101.fits.gz
    bias  sp1 60023     b1 00000102     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-b1-00000102.fits.gz
    bias  sp1 60023     b1 00000103     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-b1-00000103.fits.gz
    bias  sp1 60023     b1 00000104     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-b1-00000104.fits.gz
    bias  sp1 60023     b1 00000105     0.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-b1-00000105.fits.gz
    bias  sp1 60023     b1 00000106     0.0 /home/mejia/Research/UNAM/lvm-dr

In [22]:
fiberflats = frames_groups[(unique_groups["imagetyp"]=="flat")&(unique_groups["spec"]=="sp1")&(unique_groups["expnum"]=="00001002")]
fiberflats.pprint()

# Now that we have calibration frames, let's calibrate a fiberflat and an arc
for fiberflat_frame in fiberflats:
    fiberflat_path = fiberflat_frame["path"]
    fiberflat_camera, fiberflat_expnum = fiberflat_frame["camera"], fiberflat_frame["expnum"]
    fiberflat_mjd = fiberflat_frame["mjd"]
    fiberflat_exptime = fiberflat_frame["exptime"]
    
    imageMethod.preprocRawFrame_drp(
        in_image=fiberflat_path,
        out_image=out_main_path.format(kind="pfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        subtract_overscan=True, unit="electron", compute_error=True,
        assume_imagetyp="fiberflat", plot=2
    )
    # imageMethod.LACosmic_drp(
    #     in_image=out_main_path.format(kind="pfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
    #     out_image=out_main_path.format(kind="rfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
    #     iter=1, increase_radius=1, flim=1.3, parallel="auto"
    # )
    imageMethod.basicCalibration_drp(
        in_image=out_main_path.format(kind="pfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        out_image=out_main_path.format(kind="cfiberflat", camera=fiberflat_camera, expnum=fiberflat_expnum, ext="fits"),
        in_bias=out_calib_path.format(mjd=fiberflat_mjd, kind="mbias", camera=fiberflat_camera, exptime=fiberflat_exptime, ext="fits")
    )

imagetyp spec  mjd  camera  expnum  exptime                                           path                                         
-------- ---- ----- ------ -------- ------- ---------------------------------------------------------------------------------------
    flat  sp1 60023     b1 00001002    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-b1-00001002.fits.gz
    flat  sp1 60023     r1 00001002    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-r1-00001002.fits.gz
    flat  sp1 60023     z1 00001002    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-z1-00001002.fits.gz
[INFO]: parsed overscan region from 'BIASSEC', Y = ['1', '4080'], X = ['2042', '2080']
[INFO]: parsed data region from 'TRIMSEC', YX_i = (['1', '4080'], ['1', '2041']), YX_f = (['1', '4080'], ['2081', '4120'])
[INFO]: median counts in overscan sections {'a': 993.0, 'b': 994.0, 'c': 985.0, 'd': 993.0}
[INFO]: standard deviation in overscan

In [25]:
arcs = frames_groups[(unique_groups["imagetyp"]=="arc")&(unique_groups["spec"]=="sp1")&(unique_groups["expnum"]=="00010001")]
arcs.pprint()

for arc_frame in arcs:
    arc_path = arc_frame["path"]
    arc_camera, arc_expnum = arc_frame["camera"], arc_frame["expnum"]
    arc_mjd = arc_frame["mjd"]
    arc_exptime = arc_frame["exptime"]
    
    imageMethod.preprocRawFrame_drp(
        in_image=arc_path,
        out_image=out_main_path.format(kind="parc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
        subtract_overscan=True, unit="electron", compute_error=True, plot=2
    )
    # imageMethod.LACosmic_drp(
    #     in_image=out_main_path.format(kind="parc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
    #     out_image=out_main_path.format(kind="rarc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
    #     iter=3, increase_radius=1, flim=1.3, parallel="auto"
    # )
    imageMethod.basicCalibration_drp(
        in_image=out_main_path.format(kind="parc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
        out_image=out_main_path.format(kind="carc", camera=arc_camera, expnum=arc_expnum, ext="fits"),
        in_bias=out_calib_path.format(mjd=arc_mjd, kind="pbias", camera=arc_camera, exptime=arc_exptime, ext="fits"),
    )

imagetyp spec  mjd  camera  expnum  exptime                                           path                                         
-------- ---- ----- ------ -------- ------- ---------------------------------------------------------------------------------------
     arc  sp1 60023     b1 00010001    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-b1-00010001.fits.gz
     arc  sp1 60023     r1 00010001    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-r1-00010001.fits.gz
     arc  sp1 60023     z1 00010001    10.0 /home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/sim_2d/sdR-s-z1-00010001.fits.gz
[INFO]: parsed overscan region from 'BIASSEC', Y = ['1', '4080'], X = ['2042', '2080']
[INFO]: parsed data region from 'TRIMSEC', YX_i = (['1', '4080'], ['1', '2041']), YX_f = (['1', '4080'], ['2081', '4120'])
[INFO]: median counts in overscan sections {'a': 993.0, 'b': 994.0, 'c': 985.0, 'd': 993.0}
[INFO]: standard deviation in overscan